In [1]:
!nvidia-smi

Mon Jun 10 19:55:33 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.36                 Driver Version: 546.33       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060 Ti     On  | 00000000:01:00.0  On |                  N/A |
|  0%   56C    P5              30W / 200W |   1908MiB /  8192MiB |      7%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import pandas as pd

from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification

from utils import get_word_indicies, process_prediction, get_prediction

In [4]:
test = pd.read_csv('gt_test.csv')

test['label'] = test['label'].apply(lambda x: eval(x))
test = test.rename(columns={'label':'default_label'})

tokenizer = AutoTokenizer.from_pretrained('samolet_finetuned_0_59', model_max_length=512)
model = AutoModelForTokenClassification.from_pretrained('samolet_finetuned_0_59')

pipe = pipeline(model=model, tokenizer=tokenizer, task='ner', aggregation_strategy='average', device=0)

In [5]:
label_lst = []

for text in test['processed_text']:
    prediction = pipe(text)
    pred_labels = get_prediction(text, prediction)
    label_lst.append(pred_labels)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [11]:
test['label'] = label_lst

In [12]:
if len(test[[len(pred) != len(default) for pred, default in test[['default_label', 'label']].values]]) != 0:
    raise(ValueError('Prediction and default label length mismatch'))
    
if len(test[test['label'] != test['default_label']]) == 0:
    raise(ValueError('No predictions detected'))

In [13]:
print(len(test[test['label'] != test['default_label']]))

75


In [14]:
test[['processed_text', 'label']].to_csv('prediction.csv', index=False)